In [43]:
from pymongo import MongoClient
import pandas as pd
from getpass import getpass
from dotenv import load_dotenv
import os
import requests
import json
import plotly.graph_objects as go
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [44]:
# Hidding my token for FourSquare
load_dotenv()
token = os.getenv('token')

In [45]:
#Connectiong to Mongo DB
client = MongoClient("localhost:27017")
db = client['Ironhack']
c = db.get_collection("companies")

In [180]:
# visualizing sample
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

### Searching locations with other Gaming companies that raised over $1M
- San Francisco has 39 other companies
- New York has 30
- Los Angeles has 11

In [112]:
condition1 = {"category_code":"games_video"}
condition2 = {"funding_rounds.raised_amount":{"$gt": 1000000}}
query = {"$and":[condition1,condition2]}
projection = {"name":1, "_id":0, "offices.city":1}
games = list(c.find(query, projection))
df = pd.DataFrame(games)
df = df.explode('offices')
df.offices.value_counts()

offices
{'city': 'San Francisco'}    39
{'city': 'New York'}         30
{'city': 'Los Angeles'}      11
{'city': ''}                  8
{'city': 'Paris'}             8
                             ..
{'city': 'Boulder'}           1
{'city': 'Dundee'}            1
{'city': 'TromsÃ¸'}           1
{'city': 'Abercynon'}         1
{'city': 'Vancouver'}         1
Name: count, Length: 118, dtype: int64

#### Searching for locations with web related companies that raised over $1M
- San Francisco has 71
- New York has 65
- Palo Alto 22

In [48]:
condition1 = {"category_code":"web"}
condition2 = {"funding_rounds.raised_amount":{"$gt": 1000000}}
query = {"$and":[condition1,condition2]}
projection = {"name":1, "_id":0, "offices.city":1}
web = list(c.find(query, projection))
df = pd.DataFrame(web)
df = df.explode('offices')
df.offices.value_counts()


offices
{'city': 'San Francisco'}    71
{'city': 'New York'}         65
{'city': 'Palo Alto'}        22
{'city': 'Seattle'}          20
{'city': 'London'}           20
                             ..
{'city': 'Waltham'}           1
{'city': 'Pittsburgh'}        1
{'city': 'Burlingame'}        1
{'city': 'Lowell'}            1
{'city': 'Calabassas'}        1
Name: count, Length: 194, dtype: int64

In [93]:
condition1 = {"offices.city":"San Francisco"}
condition2 = {"funding_rounds.raised_amount":{"$gt": 1000000}}
query = {"$and":[condition1,condition2]}
pipeline = [
    {"$match": query},
    {"$unwind": "$offices"},
    {"$match": {"offices.city": "San Francisco"}},
    {"$project": {"_id": 0, "name": 1, "latitude": "$offices.latitude", "longitude": "$offices.longitude"}}
]

start = list(c.aggregate(pipeline))
start_ups  = pd.DataFrame(start)
start_ups.drop_duplicates(inplace=True)
start_ups.dropna(inplace=True)


#### Design companies locations

In [49]:
query = {"category_code":"design"}
projection = {"name":1, "_id":0, "offices.city":1}
design = list(c.find(query, projection))
df = pd.DataFrame(design)
df = df.explode('offices')
df.offices.value_counts()

offices
{'city': 'San Francisco'}    1
{'city': 'Collingwood'}      1
{'city': 'Berlin'}           1
{'city': 'Ellensburg'}       1
{'city': 'Brooklyn'}         1
{'city': 'London'}           1
Name: count, dtype: int64

## SAN FRANCISCO was the chosen location to steal a company's venue.

##### Within SAN FRANCISCO wich EXISTING offices satisfy the spatial needs for 87 employees?

In [50]:
condition1 = {"number_of_employees":{"$lt": 120}}
condition2 = {"number_of_employees":{"$gte": 90}}
condition3 = {"offices.city":"San Francisco"}
query = {"$and":[condition1,condition2,condition3]}

pipeline = [
    {"$match": query},
    {"$unwind": "$offices"},
    {"$match": {"offices.city": "San Francisco"}},
    {"$project": {"_id": 0, "name": 1, "latitude": "$offices.latitude", "longitude": "$offices.longitude", "address": "$offices.address1"}}
]

final_list = list(c.aggregate(pipeline))
final_list[0]

{'name': 'hi5',
 'latitude': 37.788668,
 'longitude': -122.400558,
 'address': '55 Second Street'}

In [51]:
# Little Cleaning
possible_venues = pd.DataFrame(final_list)
possible_venues.drop_duplicates(inplace=True)
possible_venues.dropna(inplace=True)
possible_venues.head()

,name,latitude,longitude,address
0,hi5,37.788668,-122.400558,55 Second Street
1,eBuddy,37.787683,-122.410943,555 Post St.
2,Greystripe,37.710677,-122.393230,160 Spear Street Fl 14
3,Zynga,37.765158,-122.404234,365 Vermont St.
6,99designs,37.795531,-122.400598,447 Battery St.


In [52]:
# Getting the design company position in map
condition1 = {"category_code":"design"}
condition2 = {"offices.city":"San Francisco"}
query = {"$and":[condition1,condition2]}
projection = {"_id":0, "offices.latitude":1, "offices.longitude":1, "offices.city": 1}
web = list(c.find(query, projection))
df = pd.DataFrame(web)
df = df.explode('offices')
df.offices.value_counts()

offices
{'city': 'San Francisco', 'latitude': 37.7955307, 'longitude': -122.4005983}    1
{'city': 'Collingwood', 'latitude': -37.8026587, 'longitude': 144.9868546}      1
{'city': 'Berlin', 'latitude': 52.4986199, 'longitude': 13.4469031}             1
Name: count, dtype: int64

##### Inserting the locations in the San Francisco area map for visualization.

In [110]:
import folium
# Basic map setup
map_venues2 = folium.Map(location=[possible_venues.loc[0, 'latitude'], possible_venues.loc[0, 'longitude']], zoom_start=13,tiles='cartodbpositron', control_scale=True)

for i, r in start_ups.iterrows():
    tooltip2 = f"STARTUPS: {r['name']}"
    icon2 = Icon (
        color = "green",
        icon_color = "white",
        icon = "usd",
        prefix = "fa")
    folium.Marker([r['latitude'], r['longitude']], tooltip=tooltip2, icon=icon2).add_to(map_venues2)
"""
# iterate over the rows in the DataFrame and add a marker for each possible location
for index, row in possible_venues.iterrows():
    tooltip = f"POSSIBLE ACQUISTION: {row['name']}: {row['address']}"
    icon = Icon (
        color = "blue",
        icon_color = "white",
        icon = "building ",
        prefix = "fa")
    folium.Marker([row['latitude'], row['longitude']], tooltip=tooltip, icon=icon).add_to(map_venues2)
"""
    
# add the design company to the map
icon2 = Icon (
        color = "beige",
        icon_color = "white",
        icon = "tint",
        prefix = "fa")
folium.Marker([37.7955307, -122.4005983], tooltip="DESIGN COMPANY", icon=icon2).add_to(map_venues2)

# circle desired area
folium.Circle(location=[37.7804301,-122.4103305], fill_color='green', radius=3500, weight=2, color="green", popup="Desired Area").add_to(map_venues2)
map_venues2.save('image/most_valued_startups.html')
map_venues2

In [117]:
import folium
# Basic map setup
map_venues1 = folium.Map(location=[possible_venues.loc[0, 'latitude'], possible_venues.loc[0, 'longitude']], zoom_start=13,tiles='cartodbpositron', control_scale=True)

# iterate over the rows in the DataFrame and add a marker for each possible location
for index, row in possible_venues.iterrows():
    tooltip = f"POSSIBLE ACQUISTION: {row['name']}: {row['address']}"
    icon = Icon (
        color = "blue",
        icon_color = "white",
        icon = "building ",
        prefix = "fa")
    folium.Marker([row['latitude'], row['longitude']], tooltip=tooltip, icon=icon).add_to(map_venues1)

    
# add the design company to the map
icon2 = Icon (
        color = "beige",
        icon_color = "white",
        icon = "tint",
        prefix = "fa")
folium.Marker([37.7955307, -122.4005983], tooltip="DESIGN COMPANY", icon=icon2).add_to(map_venues1)

# circle desired area
folium.Circle(location=[37.7804301,-122.4103305], fill_color='green', radius=3500, weight=2, color="green").add_to(map_venues1)
map_venues1.save('image/possible_offices.html')
map_venues1

In [54]:
def foursquare_places (venue, coordinates):
    """
    This function get resu
    args:
    :venue: String for the type of business you are searching for
    :coordinates: String for the region you want the search to be executed
    returns 
    """
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&near={coordinates}&limit=45"
    headers = {"accept": "application/json", "Authorization": token}
    retorno = requests.get(url, headers=headers).json()
    print(retorno)
    places = []
    
    for result in retorno["results"]:
        name = result["name"]
        lat = result["geocodes"]["main"]["latitude"]
        lon = result["geocodes"]["main"]["longitude"]
        places.append({"name": name, "lat": lat, "lon": lon})
    
    return places

In [55]:
"""
starbucks = foursquare_places ("starbucks", "San Francisco")
starbucks
"""

'\nstarbucks = foursquare_places ("starbucks", "San Francisco")\nstarbucks\n'

In [56]:
"""
with open("data/starbucks.json", "w") as f:
    json.dump(starbucks, f)
"""

'\nwith open("data/starbucks.json", "w") as f:\n    json.dump(starbucks, f)\n'

In [57]:
with open("data/starbucks.json", "r") as f:
    starbucks_venues = json.load(f)


In [58]:
"""
bars = foursquare_places ("bars", "San Francisco")
with open("data/bars.json", "w") as f:
    json.dump(bars, f)
bars
"""
with open("data/bars.json", "r") as f:
    bar_venues = json.load(f)

In [59]:
"""
vegan = foursquare_places ("vegan", "San Francisco")
with open("data/vegan.json", "w") as f:
    json.dump(vegan, f)
"""
with open("data/vegan.json", "r") as f:
    vegan_venues = json.load(f)


In [60]:
"""
school = foursquare_places ("school", "San Francisco")
with open("data/school.json", "w") as f:
    json.dump(school, f)
"""
with open("data/school.json", "r") as f:
    school_venues = json.load(f)

In [61]:
"""
pet = foursquare_places ("Pet Grooming Service", "San Francisco")
with open("data/pet.json", "w") as f:
    json.dump(pet, f)
"""
with open("data/pet.json", "r") as f:
    grooming_venues = json.load(f)

In [62]:
map_alltogether = folium.Map(location=[possible_venues.loc[0, 'latitude'], possible_venues.loc[0, 'longitude']], zoom_start=13,tiles='cartodbpositron')

# create a list of (latitude, longitude) tuples from your DataFrame
locations = [(venue['lat'], venue['lon']) for venue in grooming_venues]

# add a heatmap layer to the map
heatmap_layer = HeatMap(locations)
heatmap_layer.add_to(map_alltogether)

# iterate over the rows in the DataFrame and add a marker for each location
for index, row in possible_venues.iterrows():
    tooltip = f"{row['name']}: {row['address']}"
    icon = folium.Icon (
        color = "blue",
        icon_color = "white",
        icon = "building ",
        prefix = "fa")
    folium.Marker([row['latitude'], row['longitude']], tooltip=tooltip, icon=icon).add_to(map_alltogether)

# display the map
map_alltogether

In [73]:
basketball_stadium = foursquare_places ("basketball stadium", "San Francisco")
basketball_stadium[:2]

{'results': [{'fsq_id': '587d034bfad9dc1f606fc83c', 'categories': [{'id': 10051, 'name': 'Stadium', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/stadium_', 'suffix': '.png'}}], 'chains': [{'id': '62500e17-45ee-4bc8-a2cc-3a17035011ab', 'name': 'Pro NBA Arenas'}], 'distance': 2914, 'geocodes': {'main': {'latitude': 37.767924, 'longitude': -122.387478}, 'roof': {'latitude': 37.767924, 'longitude': -122.387478}}, 'link': '/v3/places/587d034bfad9dc1f606fc83c', 'location': {'address': '1 Warriors Way', 'census_block': '060750607013007', 'country': 'US', 'cross_street': '', 'dma': 'San Francisco-Oakland-San Jose', 'formatted_address': '1 Warriors Way, San Francisco, CA 94158', 'locality': 'San Francisco', 'postcode': '94158', 'region': 'CA'}, 'name': 'Chase Center Stadium', 'related_places': {'children': [{'fsq_id': '5d630ed3dec70e0008a53621', 'name': 'Bakesale Betty'}, {'fsq_id': '5d9914702ffbd200080be0c2', 'name': 'Modelo Cantina'}, {'fsq_id': '609b39f1d57bd

[{'name': 'Chase Center Stadium', 'lat': 37.767924, 'lon': -122.387478},
 {'name': 'Kezar Pavilion', 'lat': 37.767456, 'lon': -122.453888}]

In [76]:
basketball_stadium

[{'name': 'Chase Center Stadium', 'lat': 37.767924, 'lon': -122.387478},
 {'name': 'Kezar Pavilion', 'lat': 37.767456, 'lon': -122.453888},
 {'name': 'Google Cloud Courtside Lounge 25',
  'lat': 37.768163,
  'lon': -122.387778},
 {'name': 'United Club', 'lat': 37.767832, 'lon': -122.387401},
 {'name': 'Cowell Pavilion', 'lat': 37.747456, 'lon': -122.496039}]

In [150]:
map_venues3 = folium.Map(location=[possible_venues.loc[0, 'latitude'], possible_venues.loc[0, 'longitude']],control_scale=True ,zoom_start=14,tiles='cartodbpositron')
# create all the heatmaps for venues (PET, STARBUCKS, SCHOOLS, BARS, VEGAN RESTAURANTS)
grooming_locations = [(venue['lat'], venue['lon']) for venue in grooming_venues]
grooming_heatmap = HeatMap(grooming_locations, name='Pet Grooming')
grooming_heatmap.add_to(map_venues3)

starbucks_locations = [(venue['lat'], venue['lon']) for venue in starbucks_venues]
starbucks_heatmap = HeatMap(starbucks_locations, name='Starbucks')
starbucks_heatmap.add_to(map_venues3)

school_locations = [(venue['lat'], venue['lon']) for venue in school_venues]
school_heatmap = HeatMap(school_locations, name='School')
school_heatmap.add_to(map_venues3)

bar_locations = [(venue['lat'], venue['lon']) for venue in bar_venues]
bar_heatmap = HeatMap(bar_locations, name='Bars')
bar_heatmap.add_to(map_venues3)

vegan_locations = [(venue['lat'], venue['lon']) for venue in vegan_venues]
vegan_heatmap = HeatMap(vegan_locations, name='Vegan')
vegan_heatmap.add_to(map_venues3)

# add all possibel offices for takeover
for index, row in possible_venues.iterrows():
    tooltip = f"{row['name']}: {row['address']}"
    icon = folium.Icon (
        color = "blue",
        icon_color = "white",
        icon = "building ",
        prefix = "fa")
    folium.Marker([row['latitude'], row['longitude']], tooltip=tooltip, icon=icon).add_to(map_venues3)

# add the BAKETBALL stadium venues to the plot
for i in basketball_stadium:
    icon3 = folium.Icon (
        color = "green",
        icon_color = "white",
        icon = "dot-circle",
        prefix = "fa")
    folium.Marker([i['lat'], i['lon']], tooltip="BASKETBALL STADIUM", icon=icon3).add_to(map_venues3)
icon2 = Icon (
    color = "beige",
    icon_color = "white",
    icon = "tint ",
    prefix = "fa")

# add the DESIGN company to the plot
folium.Marker([37.7955307, -122.4005983], tooltip="DESIGN COMPANY", icon=icon2).add_to(map_venues3)
# circle desired area
folium.Circle(location=[37.7804301,-122.4103305], radius=3500, weight=2, color="green").add_to(map_venues3)

folium.LayerControl().add_to(map_venues3)

map_venues3.save('image/heatmap_separate_venues.html')
map_venues3


In [68]:
train_station = foursquare_places ("train station", "San Francisco")
train_station[0]

{'results': [{'fsq_id': '4ef8dcba5c5c68fa0c9920fe', 'categories': [{'id': 19047, 'name': 'Rail Station', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_', 'suffix': '.png'}}], 'chains': [], 'distance': 874, 'geocodes': {'main': {'latitude': 37.768498, 'longitude': -122.425357}, 'roof': {'latitude': 37.768498, 'longitude': -122.425357}}, 'link': '/v3/places/4ef8dcba5c5c68fa0c9920fe', 'location': {'census_block': '060750202023001', 'country': 'US', 'cross_street': 'at Dolores', 'dma': 'San Francisco-Oakland-San Jose', 'formatted_address': 'at Dolores, San Francisco, CA 94114', 'locality': 'San Francisco', 'postcode': '94114', 'region': 'CA'}, 'name': 'Muni Market & Dolores', 'related_places': {}, 'timezone': 'America/Los_Angeles'}, {'fsq_id': '4c9e1d4ed3c2b60c19bec7bc', 'categories': [{'id': 19047, 'name': 'Rail Station', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_', 'suffix': '.png'}}], 'chains': [], 'distance': 2125, '

[{'name': 'Muni Market & Dolores', 'lat': 37.768498, 'lon': -122.425357},
 {'name': 'Caltrain', 'lat': 37.775865, 'lon': -122.395185},
 {'name': 'San Francisco Caltrain Station',
  'lat': 37.776537,
  'lon': -122.394938},
 {'name': 'Caltrain', 'lat': 37.756381, 'lon': -122.392748},
 {'name': '22nd Street Caltrain Station',
  'lat': 37.757471,
  'lon': -122.392597},
 {'name': 'N-Judah 28th & Judah', 'lat': 37.761163, 'lon': -122.486471},
 {'name': 'MUNI Metro Stop - Judah & 43rd',
  'lat': 37.759259,
  'lon': -122.508205},
 {'name': 'Bayshore Caltrain Station', 'lat': 37.709174, 'lon': -122.401078},
 {'name': 'MUNI F Line Train no. 1058', 'lat': 37.774512, 'lon': -122.419763},
 {'name': 'San Francisco Paratransit Broker Atc-Intelitran',
  'lat': 37.773428,
  'lon': -122.419413},
 {'name': 'MUNI F Line Train No. 1062', 'lat': 37.781719, 'lon': -122.411372},
 {'name': 'Church Station', 'lat': 37.76758, 'lon': -122.429244},
 {'name': 'MUNI F Line Car 1077 Birmingham',
  'lat': 37.767135,
 

In [70]:
basketball_station

[{'name': 'Chase Center Stadium', 'lat': 37.767924, 'lon': -122.387478},
 {'name': 'Kezar Pavilion', 'lat': 37.767456, 'lon': -122.453888},
 {'name': 'Google Cloud Courtside Lounge 25',
  'lat': 37.768163,
  'lon': -122.387778},
 {'name': 'United Club', 'lat': 37.767832, 'lon': -122.387401},
 {'name': 'Cowell Pavilion', 'lat': 37.747456, 'lon': -122.496039}]

In [140]:
url = "https://api.foursquare.com/v3/places/search?categories=13377&near=San%20Francisco&sort=RATING&limit=3"

headers = {"accept": "application/json",
    "Authorization": token}

response = requests.get(url, headers=headers)

In [194]:
top_vegan = response.json()
top_vegan['results'][1]

{'fsq_id': '57fbb40ecd1057f331ffd3e3',
 'categories': [{'id': 13065,
   'name': 'Restaurant',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
    'suffix': '.png'}},
  {'id': 13377,
   'name': 'Vegan and Vegetarian Restaurant',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
    'suffix': '.png'}}],
 'chains': [{'id': 'e9b6ff79-62b5-4a0b-9ef6-b9bf15f3b53f',
   'name': 'sweetgreen'}],
 'distance': 2282,
 'geocodes': {'main': {'latitude': 37.787275, 'longitude': -122.398509},
  'roof': {'latitude': 37.787275, 'longitude': -122.398509}},
 'link': '/v3/places/57fbb40ecd1057f331ffd3e3',
 'location': {'address': '171 2nd St',
  'census_block': '060750615011007',
  'country': 'US',
  'cross_street': '',
  'dma': 'San Francisco-Oakland-San Jose',
  'formatted_address': '171 2nd St, San Francisco, CA 94105',
  'locality': 'San Francisco',
  'postcode': '94105',
  'region': 'CA'},
 'name': 'Sweetgreen',
 'related_places': {},
 'timez

In [156]:
all_venues = grooming_venues + bar_venues + starbucks_venues + school_venues + vegan_venues

In [149]:
map_venues5 = folium.Map(location=[possible_venues.loc[0, 'latitude'], possible_venues.loc[0, 'longitude']],control_scale=True ,zoom_start=14,tiles='cartodbpositron')
# create all the heatmaps for venues (PET, STARBUCKS, SCHOOLS, BARS, VEGAN RESTAURANTS)
all_locations = [(venue['lat'], venue['lon']) for venue in all_venues]
all_locations = HeatMap(all_locations, name='All ')
all_locations.add_to(map_venues5)


# add all possibel offices for takeover
for index, row in possible_venues.iterrows():
    tooltip = f"{row['name']}: {row['address']}"
    icon = folium.Icon (
        color = "blue",
        icon_color = "white",
        icon = "building ",
        prefix = "fa")
    folium.Marker([row['latitude'], row['longitude']], tooltip=tooltip, icon=icon).add_to(map_venues5)
"""
# add the BAKETBALL stadium venues to the plot
for i in basketball_stadium:
    icon3 = folium.Icon (
        color = "green",
        icon_color = "white",
        icon = "dot-circle",
        prefix = "fa")
    folium.Marker([i['lat'], i['lon']], tooltip="BASKETBALL STADIUM", icon=icon3).add_to(map_venues3)
icon2 = Icon (
    color = "beige",
    icon_color = "white",
    icon = "tint ",
    prefix = "fa")
"""
# add the DESIGN company to the plot
folium.Marker([37.7955307, -122.4005983], tooltip="DESIGN COMPANY", icon=icon2).add_to(map_venues5)
# circle desired area
folium.Circle(location=[37.7804301,-122.4103305], radius=3500, weight=2, color="green").add_to(map_venues5)

folium.LayerControl().add_to(map_venues5)

map_venues5.save('image/heatmap_CONDENSED_venues.html')
map_venues5

In [186]:
condition1 = {"name":"Exent"}
condition2 = {"name":"hi5"}
condition3 = {"name":"Twilio"}
condition4 = {"name":"eBuddy"}
query = {"$or":[condition1, condition2,condition3,condition4]}
projection = {"_id": 0, "name": 1, "offices.latitude": 1, "offices.longitude": 1}
pipeline = [
    {"$match": query},
    {"$unwind": "$offices"},
    {"$match": {"offices.city": "San Francisco"}},
    {"$project": {"_id": 0, "name": 1, "latitude": "$offices.latitude", "longitude": "$offices.longitude"}}
]

shortlist = list(c.aggregate(pipeline))
shortlist

[{'name': 'hi5', 'latitude': 37.788668, 'longitude': -122.400558},
 {'name': 'eBuddy', 'latitude': 37.787683, 'longitude': -122.410943},
 {'name': 'Twilio', 'latitude': 37.7898499, 'longitude': -122.4006831},
 {'name': 'Exent', 'latitude': 37.787646, 'longitude': -122.402759}]

In [199]:
url = "https://api.foursquare.com/v3/places/search?categories=11134&near=Union%20Square%2C%20San%20Francisco&limit=1"

headers = {"accept": "application/json",
    "Authorization": token}

pet_near = requests.get(url, headers=headers)
pet_near.json()

{'results': [],
 'context': {'geo_bounds': {'circle': {'center': {'latitude': 37.787963,
     'longitude': -122.40664},
    'radius': 376}}}}

In [226]:
url = "https://api.foursquare.com/v3/places/search?query=starbucks&near=685%2C%20Market%20Street%2C%20San%20Francisco&limit=6"

headers = {"accept": "application/json","Authorization": token}

coffee = requests.get(url, headers=headers)
coffee_near = coffee.json()

In [251]:
url = "https://api.foursquare.com/v3/places/search?categories=10032&near=685%2C%20Market%20Street%2C%20San%20Francisco&limit=6"
headers = {"accept": "application/json","Authorization": token}
clubs = requests.get(url, headers=headers)
club_near = clubs.json()

In [253]:
url = "https://api.foursquare.com/v3/places/search?categories=13003&near=685%2C%20Market%20Street%2C%20San%20Francisco&limit=6"
bar = requests.get(url, headers=headers)
bar_near = bar.json()

In [261]:
map_venues6 = folium.Map(location=[37.7903715,-122.4029651],control_scale=True ,zoom_start=14, tiles='openstreetmap')

# add narrow list possibel offices for takeover
for index in shortlist:
    tooltip = f"{index['name']}"
    icon = folium.Icon (
        color = "blue",
        icon_color = "white",
        icon = "building ",
        prefix = "fa")
    folium.Marker([index['latitude'], index['longitude']], tooltip=tooltip, icon=icon).add_to(map_venues6)

# add starbucks
for cup in coffee_near["results"]:
    icon2 = folium.Icon (
        color = "red",
        icon_color = "white",
        icon = "coffee ",
        prefix = "fa")
    folium.Marker([cup['geocodes']['main']['latitude'], cup['geocodes']['main']['longitude']], tooltip="STARBUCKS", icon=icon2).add_to(map_venues6)

# add night clubs
for night in club_near["results"]:
    icon3 = folium.Icon (
        color = "beige",
        icon_color = "white",
        icon = "music",
        prefix = "fa")
    folium.Marker([night['geocodes']['main']['latitude'], night['geocodes']['main']['longitude']], tooltip="Night Club", icon=icon3).add_to(map_venues6)

# add night clubs
for glass in bar_near["results"]:
    icon4 = folium.Icon (
        color = "beige",
        icon_color = "white",
        icon = "beer ",
        prefix = "fa")
    folium.Marker([glass['geocodes']['main']['latitude'], glass['geocodes']['main']['longitude']], tooltip="Bar", icon=icon4).add_to(map_venues6)

# add the BAKETBALL stadium venues to the plot
for i in basketball_stadium:
    icon3 = folium.Icon (
        color = "green",
        icon_color = "white",
        icon = "dot-circle",
        prefix = "fa")
    folium.Marker([i['lat'], i['lon']], tooltip="BASKETBALL STADIUM", icon=icon3).add_to(map_venues6)
# add the DESIGN company to the plot
folium.Marker([37.7955307, -122.4005983], tooltip="DESIGN COMPANY", icon=Icon (color = "beige",icon_color = "white",icon = "tint",prefix = "fa")).add_to(map_venues6)
# add the TOP VEGAN to the plot
folium.Marker([37.787275, -122.398509], tooltip="Sweetgreen - Second greatest rating", icon=Icon (color = "red",icon_color = "white",icon = "leaf ",prefix = "fa")).add_to(map_venues6)
# add the PET place to the plot
folium.Marker([37.787963, -122.40664], tooltip="Grooming Spot", icon=Icon (color = "red",icon_color = "white",icon = "paw",prefix = "fa")).add_to(map_venues6)
# add the FERRY TERMINAL  to the plot
folium.Marker([37.794427, -122.3945625], tooltip="Ferry Terminal", icon=Icon (color = "red",icon_color = "white",icon = "ship",prefix = "fa")).add_to(map_venues6)
# add the AIRPORT to the plot
folium.Marker([37.7254473 ,-122.2167098], tooltip="International Airport", icon=Icon (color = "red",icon_color = "white",icon = "plane",prefix = "fa")).add_to(map_venues6)
folium.Marker([37.6171039 ,-122.3856554], tooltip="International Airport", icon=Icon (color = "red",icon_color = "white",icon = "plane",prefix = "fa")).add_to(map_venues6)
# add the TRAIN STATION to the plot
folium.Marker([37.7782029 ,-122.3939985], tooltip="Train Station", icon=Icon (color = "red",icon_color = "white",icon = "train",prefix = "fa")).add_to(map_venues6)
# add the PARK to the plot
folium.Marker([37.7782029 ,-122.3939985], tooltip="Train Station", icon=Icon (color = "red",icon_color = "white",icon = "train",prefix = "fa")).add_to(map_venues6)

folium.Circle(location=[37.787646,-122.402759], fill_color="green", radius=2550, weight=2, color="green").add_to(map_venues6)

map_venues6.save('image/narrowing_venues.html')
map_venues6